In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

Merging together all the samples of a patient and create the input file of hilary

In [ ]:


def applyParallel(frame, func, silent=False, threads=None):
    """
    Parallel computing across rows of a Dataframe
    """
    max_threads = cpu_count()
    if threads is None: 
        threads = max_threads
    elif threads > max_threads: 
        threads = max_threads
    
    _dfGrouped = frame.groupby(level=0)
    with Pool(threads) as p:
        ret_list = list(tqdm(p.imap(func, _dfGrouped), total=len(_dfGrouped), disable=silent))
        
    return pd.concat(ret_list)


def hilary_parse(args):
    """
    Parsing a single row of a frame for Hilary formatting
    """
    row = args[1].loc[args[0]]
    d = dict()
    try:
        v_st, v_end = int(row.v_germline_start), int(row.v_germline_end)
        j_len = int(row.j_sequence_end - row.j_sequence_start)
        j_st = row.germline_alignment.rfind('N')+1
        d['v_sequence_alignment'] = row.sequence_alignment[v_st-1 : v_end]
        d['j_sequence_alignment'] = row.sequence_alignment[j_st : j_st + j_len]
        d['v_germline_alignment'] = row.germline_alignment[v_st-1 : v_end]
        d['j_germline_alignment'] = row.germline_alignment[j_st : j_st + j_len]
        return pd.DataFrame(d, index=[args[0]])
    except:
        print(args[0], row)
        return pd.DataFrame(index=[args[0]])

    
def hilary_preprocess(f):
    """
    Formatting the data as hilary wants (parallel computation)
    """
    df = applyParallel(f, hilary_parse)
    df['junction'] = df.index.map(f.junction)
    df['v_call'] = df.index.map(f.v_call)
    df['j_call'] = df.index.map(f.j_call)
    return df

In [3]:
metadata = pd.read_csv('metadata/metadata.tsv', sep='\t', index_col=0)
patients = set(metadata.patient)
patients

{'AT', 'D01', 'IM', 'IZ', 'MRK', 'MT'}

In [8]:
for pat in patients:
    pat_ids = metadata[metadata.patient == pat].index
    merged_fr = pd.DataFrame()
    for _id in pat_ids:
        
        if metadata.loc[_id].n_repls > 1:
            for r in range(1, metadata.loc[_id].n_repls + 1):
                fr = pd.read_csv("sequences/replicates/" + _id + '_r' + str(r) + '.tsv', sep='\t', index_col=0)
                fr = fr[fr.germline_alignment.notna()]
                fr = hilary_preprocess(fr)
                fr['sample'] = _id + '_r' + str(r)
                fr.index = fr.index.astype(str) + '_' + fr['sample']
                merged_fr = pd.concat((merged_fr, fr))
        else:
            fr = pd.read_csv("sequences/" + _id + '.tsv', sep='\t', index_col=0)
            fr = fr[fr.germline_alignment.notna()]
            fr = hilary_preprocess(fr)
            fr['sample'] = _id
            fr.index = fr.index.astype(str) + '_' + fr['sample']
            merged_fr = pd.concat((merged_fr, fr))
                
    print(pat)
    merged_fr.index.name = "sequence_id"
    #merged_fr.index = "id_" + merged_fr.index.astype(str)
    #merged_fr.index = fr['sequence_id']
    merged_fr.to_csv('lineages/seqs_in/' + pat + '.tsv', sep='\t')

100%|██████████| 2466/2466 [00:00<00:00, 2806.37it/s]


D01


100%|██████████| 4005/4005 [00:01<00:00, 2898.33it/s]


AT


100%|██████████| 1928/1928 [00:00<00:00, 3100.23it/s]


...

IM


100%|██████████| 4795/4795 [00:01<00:00, 3056.62it/s]


IZ


100%|██████████| 122/122 [00:00<00:00, 5027.81it/s]


MRK
